In [11]:
#!pip install sagemaker -U

In [5]:
import sagemaker
import boto3

sess = sagemaker.Session()

# sagemaker session bucket which is used for uploading data, models and logs
# sagemaker will automatically create the bucket if it not exists

sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set the default bucket if a buccket name is not given
    sagemaker_session_bucket = sess.default_bucket()

# Role management
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

session = sagemaker.Session(default_bucket = sagemaker_session_bucket)

print(f'sagemaker role arn:{role}')
print(f'sagemaker session region:{session.boto_region_name}')

sagemaker role arn:arn:aws:iam::396913739176:role/service-role/AmazonSageMaker-ExecutionRole-20240822T113675
sagemaker session region:us-east-1


In [6]:
from sagemaker.huggingface.model import HuggingFaceModel

# Hub model configuration <https://huggingface.co/models>
hub = {
  'HF_MODEL_ID':'distilbert-base-uncased-distilled-squad', # model_id from hf.co/models
  'HF_TASK':'question-answering'                           # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,                                                # configuration for loading model from Hub
   role=role,                                              # IAM role with permissions to create an endpoint
   transformers_version="4.26",                             # Transformers version used
   pytorch_version="1.13",                                  # PyTorch version used
   py_version='py39',                                      # Python version used
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

# example request: you always need to define "inputs"
data = {
"inputs": {
	"question": "What is used for inference?",
	"context": "My Name is Philipp and I live in Nuremberg. This model is used with sagemaker for inference."
	}
}

# request
predictor.predict(data)

------!

{'score': 0.9987204670906067, 'start': 68, 'end': 77, 'answer': 'sagemaker'}

In [7]:
predictor.predict(data)

{'score': 0.9987204670906067, 'start': 68, 'end': 77, 'answer': 'sagemaker'}

In [9]:
data = {
"inputs": {
	"question": "How is Generative AI developing peoples life?",
	"context": "Generative AI refers to a form of artificial intelligence that prioritizes the creation of original data rather than solely processing and organizing pre-existing data."
	}
}

In [10]:
predictor.predict(data)

{'score': 0.2668968439102173,
 'start': 34,
 'end': 57,
 'answer': 'artificial intelligence'}